In [1]:
import numpy as np
import tensorflow as tf 
from collections import deque
import random
import pdb
from replay_memory_Q import ReplayMemory
from q_networks import Q_Network
from test_game import test_env
from info_obj import info_holder
import os

In [2]:

import gym
game = gym.make('LunarLander-v2')
#game = gym.make('CartPole-v0')
state_dim = game.observation_space.shape[0]
action_dim = game.action_space.n
'''
game = test_env()
state_dim = 1
action_dim = 2
'''

[2016-12-30 00:16:09,789] Making new env: LunarLander-v2


'\ngame = test_env()\nstate_dim = 1\naction_dim = 2\n'

In [3]:
# info = graph, dim_s, dim_a, hid_layers, learning_rate, tau
save_dir = './Results/net/'
BUFFER_SIZE = int(1e5)
MINIBATCH_SIZE = 64
GAMMA = 0.99
LEARNING_RATE = 1e-2 #5e-5 lr/2
lr_decay_ep = 100
lr_decay_rate = 1
eps_decay_rate = 0.9
TAU = 0
ENTROPY_BETA = 0
EPSILON = 0.8
C_target_update = 16 #episode

MAX_EPISODE = int(1e4)
MAX_STEP = int(1e3)
print_ep = int(1e2)
save_ep = int(1e3)

buff = ReplayMemory(BUFFER_SIZE)
info = info_holder()
try:
    DQN
except:
    pass
else:
    DQN.reset_net()

graph = tf.Graph()
with graph.as_default():
    info.graph = graph
    info.dim_s = state_dim
    info.dim_a = action_dim
    info.hid_layers = np.array([16])
    info.learning_rate = LEARNING_RATE
    info.tau = TAU
    info.entropy_beta = ENTROPY_BETA
    DQN = Q_Network(info)

In [ ]:
eps = EPSILON
lr = LEARNING_RATE
buff.clear()
last_ep = 0
start_update = 0
with tf.Session(graph=graph) as sess:
    DQN.set_up(sess)
    sess.run(tf.global_variables_initializer())
    
    ep_reward = np.zeros(MAX_EPISODE)
    ep_success = np.zeros(MAX_EPISODE)
    ep_crash = np.zeros(MAX_EPISODE)
    batch_loss = np.zeros(MAX_EPISODE)
    #avg_var = 0
    
    for ep in xrange(MAX_EPISODE):
        s = game.reset()
        #var = 0
        for st in xrange(MAX_STEP):
            a = DQN.take_action(s.reshape(-1,info.dim_s), eps)
            s2, r, terminal, report = game.step(a)
            
            buff.add(np.reshape(s, (info.dim_s,)).astype(np.float32), np.array(a).astype(np.float32),\
                     np.array(r).astype(np.float32), np.reshape(s2, (info.dim_s,)).astype(np.float32),\
                     np.array(terminal).astype(np.float32))
            
            ep_reward[ep] += (1.*r)
            #var += np.sum(actor.give_policy(s.reshape(-1,info.dim_s))**2)
            s = s2
            if buff.size() >= MINIBATCH_SIZE:
                (s_batch, a_batch, r_batch, s2_batch, t_batch) = buff.sample_batch(MINIBATCH_SIZE)
                
                s_batch = np.array(s_batch).reshape(-1, info.dim_s)
                s2_batch = np.array(s2_batch).reshape(-1, info.dim_s)
                a_batch = np.array(a_batch)#.reshape(-1, 1)
                r_batch = np.array(r_batch)#.reshape(-1, 1)
                t_batch = np.array(t_batch)#.reshape(-1, 1)

                Q_max = DQN.target_q(s2_batch)
                Q_max = Q_max[0]
                
                R_batch = r_batch + GAMMA * Q_max * (1-t_batch) # if terminal then the second part is zero

                if(len(a_batch.shape)>1):
                    pdb.set_trace()
                    print(a_batch.shape, buff.size(), ep, st)
                batch_loss[ep], _ = DQN.train(s_batch, a_batch, R_batch, lr)
            
            if terminal:
                break
                
        if ((ep % C_target_update) == (C_target_update -1)):
            DQN.update_target()
                
        if (r >= 100):
            ep_success[ep] = 1
        elif (r <= -100):
            ep_crash[ep] = 1
        
        if ((ep % print_ep) == (print_ep-1)) or (ep == 0):
            print('Ep: %d --- Ave R: %.1f  |  loss: %.1f  |  Suc: %.2f  |  crash: %.2f  |  Max R: %.1f  |  lr:%.0f*1e-4  |  eps:%.0f*1e-2'\
                  %(ep, np.mean(ep_reward[last_ep:ep+1]), np.mean(batch_loss[last_ep:ep+1]),\
                    np.mean(ep_success[last_ep:ep+1]), np.mean(ep_crash[last_ep:ep+1]), \
                    np.amax(ep_reward[last_ep:ep+1]), 1e4*lr, 1e2*eps))
            last_ep = ep
            
        if (ep % save_ep) == (save_ep-1):
            print(' ***** Saving...  at episode %d ******' %ep)
            saver = tf.train.Saver()
            filename = save_dir + 'Episod_' + str(ep) + '.chk'
            saver.save(sess, filename)
            with open(save_dir + 'rewards.txt', 'w') as f:
                f.write('%d, , , , ,Learning rate = %f - lr decay = %f - Epsilon = %f - eps decay = %f - Buffer size = %d - Batch size = %d - hidden layers: %d - target update cycle: %d\n' \
                        %(ep, LEARNING_RATE, lr_decay_rate, EPSILON, eps_decay_rate, BUFFER_SIZE, MINIBATCH_SIZE, info.hid_layers, C_target_update) )
                for i in xrange(ep):
                    f.write('%f,      %f\n' %(ep_reward[i], batch_loss[i]))
                    
        if (ep == 0):
            try:
                with open(save_dir + 'params.txt', 'w') as f:
                    f.write('Learning rate = %f\n' %LEARNING_RATE)
                    f.write('lr decay = %f\n' %lr_decay_rate)
                    f.write('Epsilon = %f\n' %EPSILON)
                    f.write('eps decay = %f\n' %eps_decay_rate)
                    f.write('Buffer size = %d\n' %BUFFER_SIZE)
                    f.write('Batch size = %d\n' %MINIBATCH_SIZE)
                    f.write('hidden layers: %d\n' %info.hid_layers)
                    f.write('target update cycle: %d\n' %C_target_update)
            except:
                os.mkdir(save_dir)  
                with open(save_dir + 'params.txt', 'w') as f:
                    f.write('Learning rate = %f\n' %LEARNING_RATE)
                    f.write('lr decay = %f\n' %lr_decay_rate)
                    f.write('Epsilon = %f\n' %EPSILON)
                    f.write('eps decay = %f\n' %eps_decay_rate)
                    f.write('Buffer size = %d\n' %BUFFER_SIZE)
                    f.write('Batch size = %d\n' %MINIBATCH_SIZE)
                    f.write('hidden layers: %d\n' %info.hid_layers)
                    f.write('target update cycle: %d\n' %C_target_update)
            
        if (ep % lr_decay_ep) == (lr_decay_ep-1):
            lr = lr_decay_rate * lr
            eps = eps_decay_rate * eps
            
print('DONE') 
    

Ep: 0 --- Ave R: -160.8  |  loss: 155.0  |  Suc: 0.00  |  crash: 1.00  |  Max R: -160.8  |  lr:100*1e-4  |  eps:80*1e-2
Ep: 99 --- Ave R: -204.2  |  loss: 66.5  |  Suc: 0.00  |  crash: 0.99  |  Max R: -20.0  |  lr:100*1e-4  |  eps:80*1e-2
Ep: 199 --- Ave R: -162.8  |  loss: 38.1  |  Suc: 0.00  |  crash: 1.00  |  Max R: -60.8  |  lr:100*1e-4  |  eps:72*1e-2
Ep: 299 --- Ave R: -125.6  |  loss: 27.8  |  Suc: 0.00  |  crash: 1.00  |  Max R: -48.9  |  lr:100*1e-4  |  eps:65*1e-2


In [ ]:
EP = MAX_EPISODE - 1
with tf.Session(graph=graph) as sess:
    
    actor.set_up(sess)
    critic.set_up(sess)
    sess.run(tf.initialize_all_variables())
    saver = tf.train.Saver()
    filename = './saved_nets/Episod_' + str(EP) + '.chk'
    saver.restore(sess, filename)
    
    ep_reward = 0
    s = game.reset()
    for st in xrange(MAX_STEP):
        game.render()
        print(actor.give_policy(s.reshape(-1,info.dim_s)))
        A = actor.take_action(s.reshape(-1,info.dim_s))
        a = np.argmax(A)
        s2, r, terminal, report = game.step(a)
        s = s2
        ep_reward += (1.*r)
        if terminal:
            break
    print(r)
    print('reward:%f' %np.mean(ep_reward))

In [ ]:
#for i in xrange(11):
aa = xrange(11)
print('(%f, %f)'%(DQN.q_a_values(np.array(aa).reshape(-1, 1))[0],DQN.q_a_values(np.array(aa).reshape(-1, 1))[0] )


In [ ]:
print(a[0])
#game.step(a)

In [ ]:
game.step(1)

In [ ]:
a